## 二値分類での近似
入力層から中間層への積の計算はEmbeddingレイヤを導入することにより単純に行列から行を抜き出すだけの軽量な処理となった。

次は中間層以降の問題、すなわち中間層のニューロンと重み行列の積とsoftmaxレイヤの計算に対応していく。

まず、最初の問題だが、これまでは出力層は全ての単語について確率を計算するため、中間層のニューロンに巨大な行列をかけていた。今回は全ての単語ではなく、ターゲットの単語に対応する行列の列ベクトルのみをかけて、確率を計算する。すなわち多値分類だったのを二値分類問題で近似する。

多値分類のときはsoftmax関数で確率に変換したが、二値分類のときはsigmoid関数を用いる

$$
y = \frac{1}{1 + \exp(-x)}
$$

誤差関数については多値も二値も交差エントロピー誤差を用いる。

$$
L = -(t\log y + (1-t)\log(1-y))
$$

行列から列を抜き出す操作はEmbeddingレイヤで実装した。抜き出す以外に列と中間層からの出力の内積を取る処理が必要だ。なのでこれも併せてEmbeddingDotレイヤを実装する

In [1]:
import sys
sys.path.append("../../deep-learning-from-scratch-2")
import numpy as np

class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
        
    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)
        self.cache = (h, target_W)
        return out
    
    def backward(f, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

## Negative Sampling
